In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from concurrent.futures import ThreadPoolExecutor
import pandas as pd

def crawl_link(link):
    novel_data_list = []
    try:
        options = webdriver.ChromeOptions()
        options.add_argument('headless')  # 필요한 경우 창 없는 모드로 실행
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        driver.get(link)
        
        wait = WebDriverWait(driver, 10)  # 요소가 로드될 때까지 최대 10초간 대기
        
        title = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.title-wrap a'))).text
        author = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'dl.meta-author dd a strong'))).text
        intro = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#STORY-BOX p.story'))).text
        tags = [tag.text.replace('#', '') for tag in driver.find_elements(By.CSS_SELECTOR, 'div.tag-list a.novel-tag-selected')]
        
        genre_a = '기타'
        rofan_tags = set(['판타지', '로맨스', '현대', 'SF', '스포츠', '대체역사', '라이트노벨', '전쟁·밀리터리'])
        if any(tag in tags for tag in rofan_tags):
            genre_a = '판타지'
        if '패러디' in tags:
            genre_a = '패러디'
        if any(tag in ['판타지', '로맨스'] for tag in tags):
            genre_a = '로판'

        series_count = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'dl.meta-etc.meta dd:nth-of-type(1)'))).text
        view_count = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'dl.meta-etc.meta dd:nth-of-type(2)'))).text
        recommend_count = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'dl.meta-etc.meta dd:nth-of-type(3)'))).text

        novel_data = {
            'title': title,
            'author': author,
            'intro': intro,
            'genre': genre_a,
            'tags': ','.join(tags),
            'series_count': series_count,
            'view_count': view_count,
            'recommend_count': recommend_count
        }
        novel_data_list.append(novel_data)
    except Exception as e:
        print(f"Error processing link {link}: {e}")
    finally:
        driver.quit()
    return novel_data_list

def main():
    with open('문피아_소설_페이지_리스트.txt', 'r') as f:
        lines = f.readlines()
    link_list = [line.strip() for line in lines]

    results = []
    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_url = {executor.submit(crawl_link, link): link for link in link_list}
        for future in future_to_url:
            results.extend(future.result())

    munpia_novel_list = pd.DataFrame(results)
    munpia_novel_list.to_csv('munpia_novel_data.csv', encoding='utf-8-sig')

if __name__ == '__main__':
    main()


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from concurrent.futures import ThreadPoolExecutor
import pandas as pd

def crawl_link(link):
    novel_data_list = []
    try:
        options = webdriver.ChromeOptions()
        options.add_argument('headless')  # 필요한 경우 창 없는 모드로 실행
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        driver.get(link)
        
        wait = WebDriverWait(driver, 5)  # 요소가 로드될 때까지 최대 10초간 대기
        
        title = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.title-wrap a'))).text
        author = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'dl.meta-author dd a strong'))).text
        intro = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#STORY-BOX p.story'))).text
        tags = [tag.text.replace('#', '') for tag in driver.find_elements(By.CSS_SELECTOR, 'div.tag-list a.novel-tag-selected')]
        
        genre_path = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.meta-path strong'))).text

        series_count = wait.until(EC.visibility_of_element_located((By.XPATH, '//dl[@class="meta-etc meta"]/dt[text()="연재수 :"]/following-sibling::dd'))).text
        view_count = wait.until(EC.visibility_of_element_located((By.XPATH, '//dl[@class="meta-etc meta"]/dt[text()="조회수 :"]/following-sibling::dd'))).text
        recommend_count = wait.until(EC.visibility_of_element_located((By.XPATH, '//dl[@class="meta-etc meta"]/dt[text()="추천수 :"]/following-sibling::dd'))).text

        novel_data = {
            'title': title,
            'author': author,
            'intro': intro,
            'tags': ','.join(tags),
            'genre': genre_path,
            'series_count': series_count,
            'view_count': view_count,
            'recommend_count': recommend_count,
            'link': link  # 링크 추가
        }
        novel_data_list.append(novel_data)
    except Exception as e:
        print(f"Error processing link {link}: {e}")
    finally:
        driver.quit()
    return novel_data_list

def main():
    with open('문피아_소설_페이지_리스트.txt', 'r') as f:
        lines = f.readlines()
    link_list = [line.strip() for line in lines]

    results = []
    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_url = {executor.submit(crawl_link, link): link for link in link_list}
        for future in future_to_url:
            results.extend(future.result())

    munpia_novel_list = pd.DataFrame(results)
    munpia_novel_list.to_csv('munpia_novel_data.csv', encoding='utf-8-sig')

if __name__ == '__main__':
    main()


Error processing link https://novel.munpia.com/427110: Message: 
Stacktrace:
	GetHandleVerifier [0x00498923+23283]
	(No symbol) [0x0045E934]
	(No symbol) [0x00390733]
	(No symbol) [0x003D326F]
	(No symbol) [0x003D34AB]
	(No symbol) [0x0040EE42]
	(No symbol) [0x003F4464]
	(No symbol) [0x0040CB8D]
	(No symbol) [0x003F41B6]
	(No symbol) [0x003C8017]
	(No symbol) [0x003C890D]
	GetHandleVerifier [0x0058A5F3+1013699]
	GetHandleVerifier [0x00593E4C+1052700]
	GetHandleVerifier [0x0058D4B4+1025668]
	GetHandleVerifier [0x004BEA2B+179195]
	(No symbol) [0x00466833]
	(No symbol) [0x00463198]
	(No symbol) [0x00463337]
	(No symbol) [0x0045B4BE]
	BaseThreadInitThunk [0x7648FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x779480CE+286]
	RtlGetAppContainerNamedObjectPath [0x7794809E+238]

